In [1]:
import pandas
import scipy

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
infilename = "drivethrurpg-hasbro.csv"
df = pandas.read_csv(infilename).dropna(how='any')
df = df.loc[df['product'] != "ZIP"]
df.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,pdf,color,product,Greg Stafford,number of authors,number of artists
0,FlexTale Adventure Generator: Dungeons,https://www.drivethrurpg.com/en/product/481319...,536.0,5/17/24,"Any system / system-agnostic, 5e-compatible, P...","May 17, 2024",19.99,0,0,0,1,0,PDF,0,1,2
1,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",11.99,0,0,0,1,0,PDF,0,1,3
2,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",69.99,1,0,1,0,1,"Softcover, Premium Color Book (8.5"" x 11"")",0,1,3
3,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",74.99,0,1,1,0,1,"Hardcover, Premium Color Book (8.5"" x 11"")",0,1,3
4,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",39.99,1,0,0,0,1,"Softcover, Standard Color Book (8.5"" x 11"")",0,1,3


In [3]:
minPrice_df = df[["title", "price"]].groupby("title").min()
minPrice_df.head()

,price
title,
10 Feudal Japan battle maps | Asian DnD setting for fantasy RPG,9.99
100 Knick-knacks for Hollowfaust,1.25
100 Things to Find in an Alchemy Lab (5E),1.79
100(+11) Unique Puzzles for 5e!,21.85
3x the Character: Gunslingers,1.00


In [4]:
df2 = pandas.merge(df, minPrice_df, left_on="title", right_index=True).rename(columns={"price_y":"min price", "price_x":"price"})
df2.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,pdf,color,product,Greg Stafford,number of authors,number of artists,min price
0,FlexTale Adventure Generator: Dungeons,https://www.drivethrurpg.com/en/product/481319...,536.0,5/17/24,"Any system / system-agnostic, 5e-compatible, P...","May 17, 2024",19.99,0,0,0,1,0,PDF,0,1,2,19.99
1,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",11.99,0,0,0,1,0,PDF,0,1,3,11.99
2,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",69.99,1,0,1,0,1,"Softcover, Premium Color Book (8.5"" x 11"")",0,1,3,11.99
3,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",74.99,0,1,1,0,1,"Hardcover, Premium Color Book (8.5"" x 11"")",0,1,3,11.99
4,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",39.99,1,0,0,0,1,"Softcover, Standard Color Book (8.5"" x 11"")",0,1,3,11.99


# simple linear model

In [5]:
import sklearn.linear_model
lm = sklearn.linear_model.LinearRegression().fit(df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']], df2['price'])
lm

LinearRegression()

In [6]:
df2.columns

Index(['title', 'url', 'page_count', 'published_date', 'system',
       'modified_date', 'price', 'softcover', 'hardcover', 'premium', 'pdf',
       'color', 'product', 'Greg Stafford', 'number of authors',
       'number of artists', 'min price'],
      dtype='object')

In [7]:
lm.coef_

array([ 0.05929789, 12.74902341, 27.2244049 , 15.27392145,  9.88048629,
        4.87124118])

In [8]:
import statsmodels.api
import numpy
df2['constant'] = 1
#lm = statsmodels.api.OLS(numpy.asarray(df2['price']), numpy.asarray(df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']])).fit()
#lm = statsmodels.api.OLS(df2['price'].to_list(), df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']].to_dict("list")).fit()
x = df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color', 'constant']]
y = df2['price']
lm = statsmodels.api.OLS(y, x).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     724.8
Date:                Mon, 20 May 2024   Prob (F-statistic):               0.00
Time:                        16:27:21   Log-Likelihood:                -5669.3
No. Observations:                1519   AIC:                         1.135e+04
Df Residuals:                    1512   BIC:                         1.139e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0593      0.002     29.182      0.000       0.055       0.063
softcover     12.7490      0.936     13.625      0.000      10.914      14.584
hardcover     27.2244      1.056     25.776      0.000      25.153      29.296
premium       15.2739      0.772     19.783      0.000      13.759      16.788
pdf            9.8805      0.638     15.484      0.000       8.629      11.132
color          4.8712      0.913      5.334      0.000       3.080       6.663
constant      -5.8986      0.752     -7.839      0.000      -7.375      -4.423
==============================================================================
Omnibus:                      273.280   Durbin-Watson:                   1.291
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6074.147
Skew:                           0.029   Prob(JB):                         0.00
Kurtosis:                      12.796   Cond. No.                     1.17e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Why is the color coefficient negative?  Let's look at the color products...

In [9]:
df['color'].value_counts()

color
0    863
1    656
Name: count, dtype: int64

In [10]:
df.groupby(['color','hardcover']).count()['title']

color  hardcover
0      0            817
       1             46
1      0            370
       1            286
Name: title, dtype: int64

I would say that almost all physical books are in color, and it may be mixing some of the physical copy premium into the color coefficient.

In [11]:
df2['softcovern'] = df2['softcover'] * df2['page_count']
df2['hardcovern'] = df2['hardcover'] * df2['page_count']
df2['premiumn'] = df2['premium'] * df2['page_count']
df2['colorn'] = df2['color'] * df2['page_count']
df2['pdfn'] = df2['pdf'] * df2['page_count']
df2['softcoverb'] = df2['softcover'] * df2['min price']
df2['hardcoverb'] = df2['hardcover'] * df2['min price']
df2['premiumb'] = df2['premium'] * df2['min price']
df2['colorb'] = df2['color'] * df2['min price']
df2['pdfb'] = df2['pdf'] * df2['min price']
df2.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,softcovern,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb
0,FlexTale Adventure Generator: Dungeons,https://www.drivethrurpg.com/en/product/481319...,536.0,5/17/24,"Any system / system-agnostic, 5e-compatible, P...","May 17, 2024",19.99,0,0,0,...,0.0,0.0,0.0,0.0,536.0,0.00,0.00,0.00,0.00,19.99
1,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",11.99,0,0,0,...,0.0,0.0,0.0,0.0,380.0,0.00,0.00,0.00,0.00,11.99
2,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",69.99,1,0,1,...,380.0,0.0,380.0,380.0,0.0,11.99,0.00,11.99,11.99,0.00
3,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",74.99,0,1,1,...,0.0,380.0,380.0,380.0,0.0,0.00,11.99,11.99,11.99,0.00
4,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",39.99,1,0,0,...,380.0,0.0,0.0,380.0,0.0,11.99,0.00,0.00,11.99,0.00


In [29]:
df2.columns

Index(['title', 'url', 'page_count', 'published_date', 'system',
       'modified_date', 'price', 'softcover', 'hardcover', 'premium', 'pdf',
       'color', 'product', 'Greg Stafford', 'number of authors',
       'number of artists', 'min price', 'constant', 'softcovern',
       'hardcovern', 'premiumn', 'colorn', 'pdfn', 'softcoverb', 'hardcoverb',
       'premiumb', 'colorb', 'pdfb', 'priceExcess'],
      dtype='object')

In [12]:
df2['priceExcess'] = df2['price'] - df2['min price']
df3 = df2.loc[df2['priceExcess'] != 0.0]
df3.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb,priceExcess
2,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",69.99,1,0,1,...,0.0,380.0,380.0,0.0,11.99,0.00,11.99,11.99,0.00,58.00
3,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",74.99,0,1,1,...,380.0,380.0,380.0,0.0,0.00,11.99,11.99,11.99,0.00,63.00
4,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",39.99,1,0,0,...,0.0,0.0,380.0,0.0,11.99,0.00,0.00,11.99,0.00,28.00
5,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",44.99,0,1,0,...,380.0,0.0,380.0,0.0,0.00,11.99,0.00,11.99,0.00,33.00
6,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",81.98,1,0,1,...,0.0,380.0,380.0,380.0,11.99,0.00,11.99,11.99,11.99,69.99


In [40]:
lm2 = statsmodels.api.OLS(df3['priceExcess'], df3[['hardcovern', 'premiumn', 'colorn', 'constant', 'softcoverb', 'hardcoverb', 'pdfb', 'premiumb']]).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     591.5
Date:                Mon, 20 May 2024   Prob (F-statistic):          2.17e-322
Time:                        16:41:34   Log-Likelihood:                -2826.2
No. Observations:                 862   AIC:                             5668.
Df Residuals:                     854   BIC:                             5707.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
hardcovern     0.0398      0.004      8.919      0.000       0.031       0.049
premiumn       0.0975      0.006     17.006      0.000       0.086       0.109
colorn         0.0294      0.003     10.340      0.000       0.024       0.035
constant       8.8158      0.449     19.617      0.000       7.934       9.698
softcoverb    -0.2081      0.051     -4.084      0.000      -0.308      -0.108
hardcoverb    -0.1743      0.056     -3.107      0.002      -0.284      -0.064
pdfb           0.8898      0.031     28.931      0.000       0.829       0.950
premiumb      -0.2465      0.066     -3.722      0.000      -0.377      -0.117
==============================================================================
Omnibus:                       96.585   Durbin-Watson:                   1.107
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              742.385
Skew:                          -0.109   Prob(JB):                    6.21e-162
Kurtosis:                       7.541   Cond. No.                         437.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Summary
Offering any extra starts with a cost of \\$9.  If color printing is used this adds \\$0.03 per page.  

In [16]:
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'premiumn', 'pdfn', 'hardcover', 'min price']]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            priceExcess   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.917
Method:                 Least Squares   F-statistic:                              1915.
Date:                Mon, 20 May 2024   Prob (F-statistic):                        0.00
Time:                        16:27:21   Log-Likelihood:                         -2975.7
No. Observations:                 862   AIC:                                      5961.
Df Residuals:                     857   BIC:                                      5985.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0254      0.003      7.379      0.000       0.019       0.032
premiumn       0.0816      0.003     24.316      0.000       0.075       0.088
pdfn           0.0619      0.003     21.610      0.000       0.056       0.068
hardcover      8.6506      0.597     14.494      0.000       7.479       9.822
min price      0.4339      0.040     10.965      0.000       0.356       0.512
==============================================================================
Omnibus:                      103.724   Durbin-Watson:                   1.094
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              307.034
Skew:                          -0.597   Prob(JB):                     2.13e-67
Kurtosis:                       5.669   Cond. No.                         491.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
# now consider models with feature as percentage of base price
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'premiumn', 'pdfn', 'hardcover', 'min price', "softcover", "hardcovern", "softcovern", "softcoverb", "hardcoverb", "premiumb", "premium", "color", "colorb", "colorn", "pdf", "pdfn", "pdfb", "constant"]]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     269.5
Date:                Mon, 20 May 2024   Prob (F-statistic):               0.00
Time:                        16:27:22   Log-Likelihood:                -2785.5
No. Observations:                 862   AIC:                             5607.
Df Residuals:                     844   BIC:                             5693.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count    -0.0877      0.044     -1.994      0.047      -0.174      -0.001
premiumn       0.0980      0.006     16.337      0.000       0.086       0.110
pdfn           0.0017      0.003      0.679      0.497      -0.003       0.007
hardcover     -1.7651      3.137     -0.563      0.574      -7.921       4.391
min price     -0.0620      0.381     -0.163      0.871      -0.809       0.685
softcover     -9.6798      3.062     -3.161      0.002     -15.690      -3.669
hardcovern     0.1277      0.045      2.865      0.004       0.040       0.215
softcovern     0.1060      0.045      2.376      0.018       0.018       0.194
softcoverb    -0.0672      0.395     -0.170      0.865      -0.843       0.708
hardcoverb    -0.2970      0.395     -0.751      0.453      -1.073       0.479
premiumb      -0.3521      0.084     -4.176      0.000      -0.518      -0.187
premium        1.5095      1.153      1.309      0.191      -0.754       3.773
color         -0.1573      1.176     -0.134      0.894      -2.465       2.150
colorb         0.1841      0.121      1.523      0.128      -0.053       0.421
colorn         0.0139      0.008      1.742      0.082      -0.002       0.030
pdf            1.5529      0.863      1.800      0.072      -0.140       3.246
pdfn           0.0017      0.003      0.679      0.497      -0.003       0.007
pdfb           0.7596      0.073     10.409      0.000       0.616       0.903
constant      14.5118      2.923      4.965      0.000       8.775      20.248
==============================================================================
Omnibus:                       85.590   Durbin-Watson:                   1.074
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              558.716
Skew:                           0.094   Prob(JB):                    4.75e-122
Kurtosis:                       6.940   Cond. No.                     1.54e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.61e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

hardcover, softcover, premium, color, and pdf are all not significant.
The % of base cost coefficients are all significant, though the softcover % is negative.
lets take out the not significant coefficients

In [18]:
# now consider models with feature as percentage of base price
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['softcoverb', "hardcoverb", "premiumb", "pdfb", "colorn"]]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            priceExcess   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                              1174.
Date:                Mon, 20 May 2024   Prob (F-statistic):                        0.00
Time:                        16:27:22   Log-Likelihood:                         -3164.8
No. Observations:                 862   AIC:                                      6340.
Df Residuals:                     857   BIC:                                      6363.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
softcoverb     0.0532      0.054      0.990      0.323      -0.052       0.159
hardcoverb     0.3494      0.056      6.272      0.000       0.240       0.459
premiumb       0.6512      0.049     13.397      0.000       0.556       0.747
pdfb           0.9039      0.045     19.890      0.000       0.815       0.993
colorn         0.0695      0.003     20.667      0.000       0.063       0.076
==============================================================================
Omnibus:                      208.779   Durbin-Watson:                   1.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1269.657
Skew:                          -0.952   Prob(JB):                    1.98e-276
Kurtosis:                       8.633   Cond. No.                         40.1
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Summary
This model has a 96% R^2 and is both simple and intuitive.  Softcover B&W are priced at a premium over just PDF of 39+-10\% of the base price.  Hardcovers are priced at 83+-11\% of the base price.  Premium printing costs an additional 81+-10\% of the base price above that.  Color printing is priced at just \\$0.06 per page, and offering a PDF in addition to print is priced at 97+-10\% of the base price (i.e. almost exactly the PDF price without discount).

# Base price model

In [19]:
base_df = df2.loc[df2['price'] == df2['min price']]
base_df.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb,priceExcess
0,FlexTale Adventure Generator: Dungeons,https://www.drivethrurpg.com/en/product/481319...,536.0,5/17/24,"Any system / system-agnostic, 5e-compatible, P...","May 17, 2024",19.99,0,0,0,...,0.0,0.0,0.0,536.0,0.0,0.0,0.0,0.0,19.99,0.0
1,Baptism of Fire,https://www.drivethrurpg.com/en/product/470760...,380.0,4/27/24,"Dungeons & Dragons, Other OSR Games","Mar 31, 2024",11.99,0,0,0,...,0.0,0.0,0.0,380.0,0.0,0.0,0.0,0.0,11.99,0.0
10,The Spell Saying Guide,https://www.drivethrurpg.com/en/product/464773...,9.0,12/22/23,"Any system / system-agnostic, 5e-compatible","Dec 22, 2023",0.99,0,0,0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.99,0.0
11,OLDSKULL UNDERWORLD GENERATOR 1: Rats in the W...,https://www.drivethrurpg.com/en/product/479835...,260.0,5/4/24,"Dungeons & Dragons, Old-School Essentials","May 4, 2024",4.99,0,0,0,...,0.0,0.0,0.0,260.0,0.0,0.0,0.0,0.0,4.99,0.0
12,Return of the Lazy Dungeon Master,https://www.drivethrurpg.com/en/product/251431...,100.0,9/6/18,Dungeons & Dragons,"Apr 13, 2022",7.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [20]:
base_df.columns

Index(['title', 'url', 'page_count', 'published_date', 'system',
       'modified_date', 'price', 'softcover', 'hardcover', 'premium', 'pdf',
       'color', 'product', 'Greg Stafford', 'number of authors',
       'number of artists', 'min price', 'constant', 'softcovern',
       'hardcovern', 'premiumn', 'colorn', 'pdfn', 'softcoverb', 'hardcoverb',
       'premiumb', 'colorb', 'pdfb', 'priceExcess'],
      dtype='object')

In [21]:
base_m1 = statsmodels.api.OLS(base_df['price'], base_df[['page_count', "constant", "number of authors", "number of artists"]]).fit()
base_m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     224.0
Date:                Mon, 20 May 2024   Prob (F-statistic):          6.83e-100
Time:                        16:27:22   Log-Likelihood:                -2175.0
No. Observations:                 657   AIC:                             4358.
Df Residuals:                     653   BIC:                             4376.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
page_count            0.0355      0.002     21.530      0.000       0.032       0.039
constant              4.0050      0.401      9.978      0.000       3.217       4.793
number of authors     0.7186      0.115      6.242      0.000       0.493       0.945
number of artists     0.2714      0.069      3.909      0.000       0.135       0.408
==============================================================================
Omnibus:                      325.205   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8734.654
Skew:                           1.630   Prob(JB):                         0.00
Kurtosis:                      20.563   Cond. No.                         327.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
base_m = statsmodels.api.OLS(base_df['price'], base_df[['page_count', "constant"]]).fit()
base_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     531.5
Date:                Mon, 20 May 2024   Prob (F-statistic):           1.46e-86
Time:                        16:27:22   Log-Likelihood:                -2212.2
No. Observations:                 657   AIC:                             4428.
Df Residuals:                     655   BIC:                             4437.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0389      0.002     23.054      0.000       0.036       0.042
constant       5.9120      0.355     16.645      0.000       5.215       6.609
==============================================================================
Omnibus:                      217.665   Durbin-Watson:                   1.914
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5625.999
Skew:                           0.875   Prob(JB):                         0.00
Kurtosis:                      17.229   Cond. No.                         273.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
base_df['system'].value_counts()

system
5e-compatible                                                                                                              217
Dungeons & Dragons                                                                                                          37
D&D-OGL                                                                                                                     37
Basic/BECMI or OD&D                                                                                                         24
1e (AD&D)                                                                                                                   22
                                                                                                                          ... 
Other systems, 5e-compatible, Mothership                                                                                     1
Any system / system-agnostic, D&D-OGL                                                                   

Given the preponderance of Call of Cthulhu, I don't think we have enough non-Cthulhu data points to make valid inferences.

# overall model

In [24]:
pricing_m = statsmodels.api.OLS(df['price'], df2[['page_count', 'constant', 'softcovern', "hardcovern", "premiumn", "pdfn", "colorn", 'softcover', 'hardcover', 'premium', 'pdf']]).fit()
pricing_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     663.5
Date:                Mon, 20 May 2024   Prob (F-statistic):               0.00
Time:                        16:27:22   Log-Likelihood:                -5417.5
No. Observations:                1519   AIC:                         1.086e+04
Df Residuals:                    1508   BIC:                         1.092e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count    -0.0055      0.006     -0.992      0.321      -0.016       0.005
constant       3.1347      0.965      3.248      0.001       1.241       5.028
softcovern     0.0470      0.007      6.594      0.000       0.033       0.061
hardcovern     0.0668      0.008      8.864      0.000       0.052       0.082
premiumn       0.0654      0.006     10.553      0.000       0.053       0.078
pdfn           0.0428      0.005      8.138      0.000       0.032       0.053
colorn         0.0267      0.005      5.229      0.000       0.017       0.037
softcover      7.6745      0.867      8.850      0.000       5.974       9.375
hardcover     14.5640      1.235     11.797      0.000      12.142      16.986
premium        6.6170      1.055      6.271      0.000       4.547       8.687
pdf            3.7750      0.919      4.108      0.000       1.973       5.577
==============================================================================
Omnibus:                      280.249   Durbin-Watson:                   1.156
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1264.093
Skew:                           0.805   Prob(JB):                    3.20e-275
Kurtosis:                       7.169   Cond. No.                     1.94e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.94e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
pricing_m = statsmodels.api.OLS(df['price'], df2[['page_count','softcovern', "hardcovern", "premiumn", "pdfn", 'softcover', 'hardcover', 'pdf', 'color']]).fit()
pricing_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.922
Model:                            OLS   Adj. R-squared (uncentered):              0.921
Method:                 Least Squares   F-statistic:                              1981.
Date:                Mon, 20 May 2024   Prob (F-statistic):                        0.00
Time:                        16:27:22   Log-Likelihood:                         -5445.1
No. Observations:                1519   AIC:                                  1.091e+04
Df Residuals:                    1510   BIC:                                  1.096e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0098      0.004      2.782      0.005       0.003       0.017
softcovern     0.0541      0.005     11.265      0.000       0.045       0.064
hardcovern     0.0722      0.005     13.487      0.000       0.062       0.083
premiumn       0.0974      0.004     25.311      0.000       0.090       0.105
pdfn           0.0287      0.004      7.918      0.000       0.022       0.036
softcover      7.3632      0.846      8.701      0.000       5.703       9.023
hardcover     14.5535      1.256     11.588      0.000      12.090      17.017
pdf            6.6577      0.411     16.203      0.000       5.852       7.464
color          4.4487      0.781      5.695      0.000       2.917       5.981
==============================================================================
Omnibus:                      254.853   Durbin-Watson:                   1.153
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1080.010
Skew:                           0.744   Prob(JB):                    3.01e-235
Kurtosis:                       6.854   Cond. No.                     1.61e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.61e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
df2[['page_count','softcovern', "hardcovern", "premiumn", "pdfn", 'softcover', 'hardcover', 'pdf', 'color']].corr()

,page_count,softcovern,hardcovern,premiumn,pdfn,softcover,hardcover,pdf,color
page_count,1.000000,0.195498,0.384183,0.184634,0.784693,-0.113859,0.241547,-0.016516,0.138209
softcovern,0.195498,1.000000,-0.232191,0.083704,-0.003331,0.725205,-0.266260,-0.212635,0.377706
hardcovern,0.384183,-0.232191,1.000000,0.382099,0.096948,-0.320173,0.872046,-0.194785,0.417597
premiumn,0.184634,0.083704,0.382099,1.000000,0.023474,-0.008043,0.333987,-0.143440,0.347782
pdfn,0.784693,-0.003331,0.096948,0.023474,1.000000,-0.191545,0.014143,0.476588,-0.103451
softcover,-0.113859,0.725205,-0.320173,-0.008043,-0.191545,1.000000,-0.367151,-0.293207,0.444459
hardcover,0.241547,-0.266260,0.872046,0.333987,0.014143,-0.367151,1.000000,-0.223365,0.458660
pdf,-0.016516,-0.212635,-0.194785,-0.143440,0.476588,-0.293207,-0.223365,1.000000,-0.368230
color,0.138209,0.377706,0.417597,0.347782,-0.103451,0.444459,0.458660,-0.368230,1.000000


In [27]:
base_df.loc[base_df['price'] < 9.20]

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb,priceExcess
10,The Spell Saying Guide,https://www.drivethrurpg.com/en/product/464773...,9.0,12/22/23,"Any system / system-agnostic, 5e-compatible","Dec 22, 2023",0.99,0,0,0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.99,0.0
11,OLDSKULL UNDERWORLD GENERATOR 1: Rats in the W...,https://www.drivethrurpg.com/en/product/479835...,260.0,5/4/24,"Dungeons & Dragons, Old-School Essentials","May 4, 2024",4.99,0,0,0,...,0.0,0.0,0.0,260.0,0.0,0.0,0.0,0.0,4.99,0.0
12,Return of the Lazy Dungeon Master,https://www.drivethrurpg.com/en/product/251431...,100.0,9/6/18,Dungeons & Dragons,"Apr 13, 2022",7.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
20,"Deck of Ensorcelled Weapons, For Use With 5E A...",https://www.drivethrurpg.com/en/product/478852...,52.0,5/16/24,5e-compatible,"May 16, 2024",4.95,0,0,0,...,0.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0,4.95,0.0
24,Deathbringer RPG,https://www.drivethrurpg.com/en/product/396879...,4.0,5/17/22,"Other systems, Dungeons & Dragons, Old-School ...","May 28, 2022",3.99,0,0,0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,Ironwic 5e Mystery #3: The Norwood Builder,https://www.drivethrurpg.com/en/product/478256...,24.0,4/23/24,D&D-OGL,"Apr 22, 2024",4.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1594,"RS1 Fang, Faith, and Legerdemain",https://www.drivethrurpg.com/en/product/422462...,64.0,1/7/23,"Basic/BECMI or OD&D, Labyrinth Lord, Old-Schoo...","Jan 7, 2023",6.95,0,0,0,...,0.0,0.0,0.0,64.0,0.0,0.0,0.0,0.0,6.95,0.0
1597,Pirate Adventurers (5E),https://www.drivethrurpg.com/en/product/172599...,9.0,2/1/16,5e-compatible,"Jul 12, 2020",1.95,0,0,0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,1.95,0.0
1598,The Space Hack,https://www.drivethrurpg.com/en/product/187763...,14.0,7/10/16,Classic D&D/AD&D,"Jul 10, 2016",2.48,0,0,0,...,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,2.48,0.0


In [28]:
base_df.loc[base_df['price'] < 9.20].to_csv("9_dollar-titles.csv", index=False)